<a href="https://colab.research.google.com/github/MosaicGroupCMU/African-MRS-Tutorials/blob/main/Google-Colab/Answers/3_Quantum_Espresso_H2O_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to geometry optimization of molecular water using Quantum ESPRESSO

Contributors: [Seda Oturak](https://github.com/sedaoturak), [Ismaila Dabo](https://scholar.google.com/citations?user=rN299m0AAAAJ&hl=en), [Jessica Wen](https://github.com/JessicaWen-PhD), [Cierra Chandler](https://github.com/Cierra-Chandler), [Henry Eya](https://github.com/Henrynweya)



# Install libraries, environment, and Quantum ESPRESSO

This part installs libraries for numerical calculations and plotting.

In [15]:
# load plotting libraries
import matplotlib.pyplot as plt

# load numerical libraries
import numpy as np

Quantum ESPRESSO is a plane wave code, which uses Fourier transforms to solve equations in plane wave space. This part installs libraries for fast Fourier transforms (FFTs).

In [16]:
# eliminate text output during installation
%%capture

# install mathematical libraries to peform fast Fourier transforms
# (the exclamation mark means that the command is run under Linux)
! apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc

The Atomic Simulation Environment (ASE) is a set of tools for running, visualizing, and analyzing simulations. This part installs ASE.

In [17]:
# eliminate text output during installation
%%capture

# install the Atomic simulation environment
# ! apt install ase
! pip install git+https://gitlab.com/ase/ase

Compiling Quantum ESPRESSO from scratch would take a long time. This part uploads pre-compiled executable files (`.x` extension) and additional files containing the pseudopotentials.

In [18]:
# eliminate text output during installation
%%capture

# navigate to main directory named '/content/'
%cd /content/

# download the pre-compiled files in compressed format (under Linux)
! wget 'https://docs.google.com/uc?export=download&id=1kw_CJMjP6ggDZXDNp5phAqCPpoe2WXCA' -O qe-lite.tgz
#! wget 'https://drive.google.com/uc?export=download&id=1cGNP9ea2ty1QtztocZGniJx9eGDXqGr1' -O qe-lite.tgz

# unpack the compressed files (under Linux)
! tar -xvzf qe-lite.tgz

# clean up some files
! rm -rf sample_data qe-lite.tgz

# Prepare Quantum ESPRESSO input file

This is where we start setting up the data and information for our Quantum ESPRESSO calculations.

As always, we first need to start with the Self-Consistent Field (SCF) calculations. The parameters in the Quantum ESPRESSO input file requires some playing around to achieve the best trade-off between accuracy and run-time.

This calculation is for a water molecule. The definition of the input parameters of the `pw.x` executable can be found at `www.quantum-espresso.org/Doc/INPUT_PW.html`.

In [19]:
# create calculation folder and navigate into it
%mkdir -p /content/water
%cd /content/water/

# create input and write it into the file h2o.scf.in
qe_input = """
&control
  prefix='water',
  pseudo_dir = '/content/qe-lite/pseudopotentials',
  outdir='/content/water',
  verbosity = 'high'
/
&system
  ibrav = 0,
  nat = 3,
  ntyp = 2,
  ecutwfc = 30.0,
  ecutrho = 240.0,
  occupations  = 'smearing'
  degauss      = 0.001
  smearing     = 'marzari-vanderbilt'
/
&electrons
  conv_thr = 1e-8,
/
ATOMIC_SPECIES
O 15.999 O.UPF
H 1.008 H.UPF

K_POINTS automatic
1 1 1 0 0 0

CELL_PARAMETERS angstrom
10.00000000000000 0.00000000000000 0.00000000000000
0.00000000000000 10.00000000000000 0.00000000000000
0.00000000000000 0.00000000000000 10.00000000000000

ATOMIC_POSITIONS angstrom
O 0.0000000000 0.0000000000 0.2981545000
H 0.0000000000 0.7632390000 -0.2981545000
H 0.0000000000 -0.7632390000 -0.2981545000

"""

with open("h2o.scf.in", "w") as f:
    f.write(qe_input)

# print the content of the input file (under Linux)
! cat h2o.scf.in

/content/water

&control
  prefix='water',
  pseudo_dir = '/content/qe-lite/pseudopotentials',
  outdir='/content/water'
/
&system
  ibrav = 0,
  nat = 3,
  ntyp = 2,
  ecutwfc = 30.0,
  ecutrho = 240.0,
  occupations  = 'smearing'
  degauss      = 0.001
  smearing     = 'marzari-vanderbilt'
/
&electrons
  conv_thr = 1e-8,
/
ATOMIC_SPECIES
O 15.999 O.UPF
H 1.008 H.UPF

K_POINTS automatic
1 1 1 0 0 0

CELL_PARAMETERS angstrom
10.00000000000000 0.00000000000000 0.00000000000000
0.00000000000000 10.00000000000000 0.00000000000000
0.00000000000000 0.00000000000000 10.00000000000000

ATOMIC_POSITIONS angstrom
O 0.0000000000 0.0000000000 0.2981545000
H 0.0000000000 0.7632390000 -0.2981545000
H 0.0000000000 -0.7632390000 -0.2981545000



Use [ASE tools](https://www.google.com/url?q=https%3A%2F%2Fwiki.fysik.dtu.dk%2Fase%2Fase%2Fio%2Fformatoptions.html%23ase.io.espresso.read_espresso_in) to extract information from Quantum ESPRESSO input and visualize the crystal.

In [20]:
import ase.io.espresso
from ase import Atoms
from ase.visualize import view
from ase.build import make_supercell
from ase.build import bulk

# extract unit cell information from input file using ASE
input_file = ase.io.espresso.read_espresso_in('h2o.scf.in')
water = Atoms(input_file)

# create a supercell (3 × 3 × 3) using ASE
multiplier = np.identity(3) * 3
water_supercell = make_supercell(water,multiplier)

# visualize the supercell
view(water_supercell, viewer='x3d')

HINT: if you're getting an AttributeError where the 'NoneType' object has no attribute 'append', it's probably because you're not quite formatting your input file correctly. For example, you might be missing commas.

# Run Quantum ESPRESSO using input file

Make sure that the pseudopotential file is present in the working directory. It's time to start the self-consistent field calculation, which is often the longest part of any calculations you do. This should produce the potential energy.

### What is the Self-Consistent Field (SCF) calculation?

The SCF calculation is an iterative computational method used to solve the many-body Schrödinger equation for electrons in atoms and molecules.

It starts with an initial guess for the electron distribution and iteratively refines it until convergence is reached. Each electron is treated as if it moves in the average field created by all other electrons, simplifying the complex many-body problem (mean-field approximation).

The calculation aims to minimise the total energy of the system with respect to the electron density, thus solving for molecular orbitals and their energies by using (in this case) plane wave basis sets to represent the electron wavefunctions.

The iteration stops when the change in energy or electron density between successive steps become sufficiently small (given by the convergence threshold).

### What are the pseudopotential files used for?

The pseudopotential files replace the full electron system with an effective potential that accounts for the core electrons, so that we don't need to calculate where every single electron in the system goes. This reduces computational complexity. Well-designed pseudopotentials aim to balance computational efficiency with accuracy across different chemical environments.

You can read more about choosing pseudopotentials on the [VASP wiki](https://www.vasp.at/wiki/index.php/Choosing_pseudopotentials).

In [21]:
# run the pw.x executable using h2o.scf.in to create h2o.scf.out
! /content/qe-lite/bin/pw.x < h2o.scf.in > h2o.scf.out

# print the content of the output file (under Linux)
! cat h2o.scf.out


     Program PWSCF v.7.4 starts on 10Dec2024 at  3:47: 4 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Serial version
     4822 MiB available memory on the printing compute node when the environment starts

     Waiting for input...
     Reading input from standard input

     Current dimensions of program PWSCF are:
     Max number of different atomic species (ntypx) = 10
     Max number of k-points (npk) =  40000
     Max angular momentum in pseudopotentials (lmaxx) =  4
     Message from routine setup:
     using ibrav=0 wi

In [22]:
# first method: extract total energies in rydberg during the self-consistent-field calculation (under Linux)
! grep -e "total energy  " -e "estimated" h2o.scf.out

     total energy              =     -34.46638195 Ry
     estimated scf accuracy    <       0.45980759 Ry
     total energy              =     -34.57693590 Ry
     estimated scf accuracy    <       0.43521711 Ry
     total energy              =     -34.65647122 Ry
     estimated scf accuracy    <       0.00394257 Ry
     total energy              =     -34.65736206 Ry
     estimated scf accuracy    <       0.00137293 Ry
     total energy              =     -34.65726977 Ry
     estimated scf accuracy    <       0.00025543 Ry
     total energy              =     -34.65733619 Ry
     estimated scf accuracy    <       0.00002315 Ry
     total energy              =     -34.65734146 Ry
     estimated scf accuracy    <       0.00000278 Ry
     total energy              =     -34.65734191 Ry
     estimated scf accuracy    <       0.00000012 Ry
     total energy              =     -34.65734191 Ry
     estimated scf accuracy    <       0.00000001 Ry
!    total energy              =     -34.65734

In [23]:
# define physical constants for unit conversion
from scipy.constants import physical_constants
ha_in_ev = physical_constants["Hartree energy in eV"][0]
ry_in_ev = ha_in_ev / 2.

# second method: extract total energy at the end of the self-consistent calculation (using ASE)
output = ase.io.read("/content/water/h2o.scf.out")
total_energy = output.get_total_energy()
print("Energy = %.8f Ry " % total_energy)
print("Energy = %.8f eV " % ( total_energy / ry_in_ev ) )

Energy = -471.53711737 Ry 
Energy = -34.65733889 eV 


# Post-processing

We can now extract the charge density from the SCF calculation using the post-processing executable (pp.x) in Quantum ESPRESSO. This requires writing another shorter input file. This must be in the same folder as the output folder from your pw.x calculation.

The pp.x executable extracts the converged wavefunctions and other relevant information to calculate the desired quantity, in this case the electron charge density. The calculated charge density is then written to a file ("filplot") for plotting and further analysis.

Post-processing inputs consist of two namelists: &INPUTPP and &PLOT.

*   &INPUTPP contains the variables for data extraction and calculation.
*   &PLOT specifies the output format and plotting parameters.

The resulting charge density integrates to the number of electrons, not the total charge. You can read more about pp.x in the [Quantum ESPRESSO documentation](https://www.quantum-espresso.org/Doc/INPUT_PP.html).


In [26]:
# Prepare post-processing input
input_pp = """
&inputpp
    prefix = 'water'
    outdir = '/content/water'
    filplot = 'water_charge_density'
/
&plot
    nfile = 1
    filepp(1) = 'water_charge_density'
    weight(1) = 1.0
    iflag = 3
    output_format = 6
    fileout = 'water_charge_density.cube'
/
"""

# Save the post-processing input file
with open("water_pp.in", "w") as file:
    file.write(input_pp)

# Run post-processing
!/content/qe-lite/bin/pp.x -inp water_pp.in > water_pp.out

print("Charge density calculation completed. Check water_charge_density.cube for the output.")

/bin/bash: line 1: /content/qe-lite/bin/pp.x: No such file or directory
Charge density calculation completed. Check water_charge_density.cube for the output.


# Visualisation

We can now use your favourite plotting software to visualise the charge density. In this case, we will use the matplotlib and numpy libraries.

In [27]:
# Load the .cube file
with open('water_charge_density.cube', 'r') as f:
    lines = f.readlines()

# Skip header and load density data
density_data = np.array([float(x) for line in lines[6:] for x in line.split()])
grid_shape = (50, 50, 50)  # Adjust based on the file
density_data = density_data.reshape(grid_shape)

# Plot a slice of the charge density
plt.imshow(density_data[:, :, grid_shape[2]//2], cmap='viridis')
plt.colorbar(label='Charge Density')
plt.title('Charge Density Slice')
plt.xlabel('x-axis')
plt.ylabel('y-axis')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'water_charge_density.cube'